<a href="https://colab.research.google.com/github/mesnico/DTfH-Laboratory/blob/main/image_similarity_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratory on Image Representations for Similarity Search
-----------------

You'll learn to:

*   Represent images using features extracted from pretrained deep neural networks.
*   Search images by visual similarity.
*   Search images using textual descriptions.

## Getting Started

First of all, we need to download and unzip the image dataset, and install some Python dependencies.

We will use  [MIRFlickr25k](https://press.liacs.nl/mirflickr/mirdownload.html), which contains photographs downloaded from the popular Flickr website.

Run the following for getting the environment ready! This could take some minutes...



In [ ]:
!gdown --id 1DvjTCh78h1vmjNe7beZuJOllYeHREl6V
!unzip mirflickr25k.zip

Next, we install some python dependency and we import the needed python packages

In [ ]:
!pip install transformers

import numpy as np
import pandas as pd

import torch
from torch.nn import functional as F
import transformers
from PIL import Image

from pathlib import Path
from tqdm.auto import tqdm

import matplotlib
import matplotlib.pyplot as plt
from skimage.transform import resize
import requests

import random
import os

%matplotlib inline
random.seed(42)

euclidean_distance = torch.cdist

In [ ]:
# define the function for computing the k-NN from the distances
def k_nearest_neighbors(distances, k=5):
  nq, ndb = distances.shape

  sorted_distances = distances.argsort(axis=1)  # sort the scores ascending, for each query
  topk = sorted_distances[:, :k]  # get **indices** of the topk images for each row
  topk_distances = distances[np.arange(nq)[:, None], topk]  # use the indices to get the topk scores (magic slicing version)
  # topk_scores = np.concatenate([scores[i, topk[i]] for i in range(nq)])  # get topk scores (comprehensible version)
  return topk, topk_distances

# define an helper function to view the results
def show_images(urls, figsize=None):
  n_images = len(urls)
  fig, axes = plt.subplots(1, n_images, figsize=figsize)
  for ax, url in zip(axes, urls):
    image_np = np.asarray(Image.open(requests.get(url, stream=True).raw))
    # image_np = unpad_image(image_np)
    image_np = resize(image_np, (400, 300))
    ax.set_ylabel(f'Query')
    ax.imshow(image_np)
    ax.set_xticks([])
    ax.set_yticks([])
  
  return fig

## Data Loading

Let's inspect the data.

In [ ]:
image_paths = Path('mirflickr').rglob('*.jpg')
image_paths = sorted(image_paths)
image_paths[:5]

In [ ]:
fig, axes = plt.subplots(5, 10, figsize=(20, 10))
for ax, image_path in zip(axes.flatten(), image_paths):
  image_np = plt.imread(image_path)
  image_np = resize(image_np, (400, 300))
  ax.imshow(image_np)
  ax.axis('off')
plt.subplots_adjust(wspace=0, hspace=0)

## Part 1 - Image Retrieval by Visual Similarity

Consider a (potentially large) database of images and a set of query images.
Our goal is to retrieve images from the database that are visually similar to the queries.

Let's first select some images among which we will search. We will define our queries later.

In [ ]:
ndb = 5000 # number of samples in the database to consider

selected_image_paths = random.sample(image_paths, ndb)
db_image_paths = selected_image_paths[:ndb]
db_image_paths[:5]

### Prepare the deep neural network!

We will use a pretrained model to extract features from images and use them as compact image representations.
The choice of the model is essential to get good representations for your target task (e.g. a model trained on classifying dog breeds will have a hard time creating good representations for images of flowers).

In this laboratory, we will use the models from the [HuggingFace](https://huggingface.co/) repository. It collects a bunch of different models pre-trained on [ImageNet-1k](https://www.image-net.org/download.php), the dataset on which deep networks for image classification are trained.

In particular, we will use one of the most recent state-of-the-art deep neural networks as image feature extractor: [ResNet](https://huggingface.co/docs/transformers/v4.19.2/en/model_doc/resnet).


In [ ]:
# a model pretrained on ImageNet-21k
model = transformers.ResNetModel.from_pretrained("microsoft/resnet-50")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

preprocess = transformers.AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")

`preprocess` is a module that performs some preprocessing on the image before feeding it to the deep neural network (scale, normalization).

`model` is the actual deep neural network (our ResNet) that we will use for extracting the features.

### Feature extraction function

Let's prepare a function `extract_features` that extracts the features from a set of images given as input, using the pre-defined deep neural network.

We hide the internals of this function, as there are some unimportant details. Feel free to unhide it if you want to know more.

However, run it before moving on!

In [ ]:
def extract_features(images, normalize=True):
  features = []

  # we repeat the extraction for each image in the given list of images
  for image in tqdm(images):

    # open the image
    image = Image.open(image)

    # perform some preprocessing (scale, normalization) on the image
    inputs = preprocess(image, return_tensors="pt")
    inputs = inputs.to(device)

    # extract the features from the image using our deep neural network
    with torch.no_grad():
      outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state

    # save the features in a list that we will return
    features.append(last_hidden_states.cpu())

  # do some post-processing on the features before returning them
  features = torch.cat(features, dim=0)
  features = features.mean(dim=[2,3])
  if normalize:
    features = F.normalize(features)

  return features

### Let's go with feature extraction!

Now, we can call this nice `extract_features` function on the list of images that we have chosen from the dataset.

Notice that the output shape is a table having a number of rows equal to the number of images, and a number of column equal to the number of dimensions of our feature. In practice, for each image in the dataset we have a 2048-dimensional feature.

In [ ]:
dataset_features = extract_features(db_image_paths)
print(dataset_features.shape)

### Choose some query images

First of all, let's select some query images for which we want to retrieve similar images in our MIRFlickr25k database.

We will use some image taken from the web as queries (you can experiment with any image URL you find on the web!).

In [ ]:
queries_urls = [
        'http://farm4.staticflickr.com/3472/3208721055_d4d9ba651b_z.jpg',
        'http://farm6.staticflickr.com/5267/5635183343_5aff7a08af_z.jpg',
        'https://farm7.staticflickr.com/6168/6148670625_1caaf41c72_z.jpg',
        'http://farm9.staticflickr.com/8233/8471600250_d2ab877363_z.jpg'
]

# let's display them
fig, axes = plt.subplots(1, len(queries_urls), figsize=(10, 5))
for ax, url in zip(axes.flatten(), queries_urls):
  image_np = np.asarray(Image.open(requests.get(url, stream=True).raw))
  image_np = resize(image_np, (400, 300))
  ax.imshow(image_np)
  ax.axis('off')
plt.subplots_adjust(wspace=.1, hspace=0)

Now we can produce the features for the queries that we have just chosen.

We do this exactly as before: we call our nice function `extract_features`, which uses our nice deep neural network for obtaining features from the query images.

In [ ]:
# download the query images from the web
queries = [requests.get(url, stream=True).raw for url in queries_urls]

# extract features of queries
queries_features = extract_features(queries)
print(queries_features.shape)

### The core of image similarity search

Once we have feature vectors representations, we can search similar representations by comparing the features vectors instead of pixels.

We will compare feature vectors using the Euclidean distance
$$
d = \sqrt{(x_1-y_1)^2 + (x_2-y_2)^2 + \dotso + (x_n-y_n)^2},
$$
where $\{x_1, x_2, ... x_n\}$ are the coordinates of the first feature and $\{y_1, y_2, ... y_n\}$ those of the second.


In [ ]:
distances = euclidean_distance(queries_features, dataset_features)

Once we have the distances, we can compute the k-nearest-neighbors for each query!
The function `k_nearest_neighbors` performs exactly this.

1. It sorts the distances from the smallest to the largest, for each query.
2. It takes the first $k$ features as a result, again for each query.

In [ ]:
k = 5 # how many neighbors for each query
topk, topk_distances = k_nearest_neighbors(distances, k)
print(topk_distances)

### Look at the results!

Let's finally view the results.

In [ ]:
nq, ndb = distances.shape

# show queries
fig = show_images(queries_urls, figsize=(3*nq, 10))

# show topk similar 
fig, axes = plt.subplots(k, nq, figsize=(fig.get_figwidth(), 4*k))
for j in range(k):
  axes[j, 0].set_ylabel(f'Rank #{j}')
  for i in range(nq):
    image_np = plt.imread(db_image_paths[topk[i, j]])
    image_np = resize(image_np, (400, 300))
    axes[j, i].imshow(image_np)
    axes[j, i].set_title('dist = {:.2f}'.format(topk_distances[i, j]))
    axes[j, i].set_xticks([])
    axes[j, i].set_yticks([])

## Part 2 - Text to Image Retrieval

In this second part, instead, we will try to retrieve images using natural texts as a query.

We need:
- a feature extractor for the _images_ (our _database_)
- a feature extractor for the _texts_ (our _queries_)

We will use the [CLIP](https://huggingface.co/docs/transformers/v4.19.2/en/model_doc/clip) deep neural network, which implements both the feature extractors.

Let's initialize CLIP.

In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

### Image and text representations

Now we define some nice helper functions:
- the `extract_features_from_images`, which extracts representations from the images;
- the `extract_features_from_texts`, which extracts representations from the images.

We hide the internals of these functions, as there are some unimportant details. Feel free to unhide it if you want to know more.

However, run the following block before moving on!

In [ ]:
def extract_features_from_images(images):
  features = []

  # we repeat the extraction for each image in the given list of images
  for image in tqdm(images):

    # open the image
    image = Image.open(image)

    # perform some preprocessing (scale, normalization) on the image
    inputs = processor(images=image, return_tensors="pt")
    inputs = inputs.to(device)

    # extract the features from the image using our CLIP deep neural network
    with torch.no_grad():
      feature = model.get_image_features(**inputs)

    # save the features in a list that we will return
    features.append(feature)

  # do some post-processing on the features before returning them
  features = torch.cat(features, dim=0)
  features = F.normalize(features)

  return features
  

def extract_features_from_texts(texts):

  # preprocess the words of the text
  inputs = tokenizer(texts, padding=True, return_tensors="pt")
  inputs = inputs.to(device)

  # extract the features from the text using the CLIP deep neural network
  with torch.no_grad():
    features = model.get_text_features(**inputs)

  # do some post-processing on the features before returning them
  features = F.normalize(features)
  return features

###Let's go with feature extraction!

Now, call the `extract_features_from_images` function to extract the image features from the whole dataset.

Look at the dimensionality of the extracted features. For each of the image in the database, we have a 512-dimensional feature (for ResNet it was 2048).

In [ ]:
image_dataset_features = extract_features_from_images(db_image_paths)
print(image_dataset_features.shape)

Then, we define some textual queries and we extract their features using this `extract_features_from_texts` function.

Note that the dimensionality of the textual features is again 512, as the image features! We can therefore compute the Euclidean distance between them.

In [ ]:
textual_queries = [
                   'A person is playing sport',
                   'A picture of an old man',
                   'A view of some mountains',
                   'There are some people in a room with a laptop'
]

textual_queries_features = extract_features_from_texts(textual_queries)
print(textual_queries_features.shape)

### The core of text-to-image similarity search

As in Part 1, we compute the Euclidean distance between the queries (that this time come from texts!) and the images in the dataset.

In [ ]:
distances = euclidean_distance(textual_queries_features, image_dataset_features)

Now, we can reuse the `k_nearest_neighbors` function already used in Part 1, as is, for searching image representations more similar to text representations!

Therefore, remember what we are doing under the hood:
1. We sort the distances from the smallest to the largest, for each query.
2. We take the first $k$ features as a result, again for each query. 

In [ ]:
k = 10
topk, topk_distances = k_nearest_neighbors(distances, k)

### Look at the results!

Let's finally view the results.

In [ ]:
nq, ndb = distances.shape

# show topk similar
fig, axes = plt.subplots(k, nq, figsize=(18, 4*k))
for j in range(k):
  axes[j, 0].set_ylabel(f'Rank #{j}')
  for i in range(nq):
    if j == 0:
      axes[0, i].set_title(textual_queries[i])
    image_np = plt.imread(db_image_paths[topk[i, j]])
    # image_np = unpad_image(image_np)
    image_np = resize(image_np, (400, 300))
    axes[j, i].imshow(image_np)
    axes[j, i].xaxis.set_label_position('top')
    axes[j, i].set_xlabel('dist = {:.2f}'.format(topk_distances[i, j]))
    axes[j, i].set_xticks([])
    axes[j, i].set_yticks([])